# Plotting live data from Cyton Board using Python

## Notes:
1. This notebook is being run on a Windonws 10 Laptop, using a Python3 environment.
2. After you installed all the [dependencies](http://docs.openbci.com/OpenBCI%20Software/05-OpenBCI_Python) correctly you can start usign this tutorial
2. Start this notebook from a command prompt with sudo or as administrator.

### Importing necessary packages:
The packages used for this tutorial are in the requirements.txt file. You can simply download that file and install the requirements by typying "pip install -r requirements.txt" on the command line.

If you are having trouble with the plugin_interface you might need to use [this workaround](https://github.com/OpenBCI/OpenBCI_Python/issues/91#issuecomment-412817422)

#### First let's import the necessary packages


In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import collections
%matplotlib inline

import sys; sys.path.append('..') # help python find cyton.py relative to scripts folder
from openbci import cyton as bci
import logging
import time
import datetime as dt
from scipy import signal

#### Now let's try to run the [test code](https://github.com/OpenBCI/OpenBCI_Python/blob/master/scripts/test.py) from the OpenBCI-Python repo

We are doing this to make sure all the dependencies are installed correctly. When working this code should print the raw data of the board. If everything is working correctly you can press crtl+C to end the script.

In [5]:
def printData(sample):
    # os.system('clear')
    print("----------------")
    print("%f" % (sample.id))
    print(sample.channel_data)
    print(sample.aux_data)
    print("----------------")

if __name__ == '__main__':
    port = 'COM5'
    baud = 115200
    logging.basicConfig(filename="test.log", format='%(asctime)s -%(levelname)s : %(message)s', level=logging.DEBUG)
    logging.info('--------------LOG START ------------')
    board = bci.OpenBCICyton(port=port, scaled_output=False, log=True)
    print("Board Instantiated")
    board.ser.write(str.encode('v'))
    time.sleep(10)
    board.start_streaming(printData)
    board.print_bytes_in()

Connecting to V3 at port COM5
Serial established...
�OpenBCI V3 8-16 channel
On Board ADS1299 Device ID: 0x3E
LIS3DH Device ID: 0x33
Firmware: v3.1.1
$$$
Board Instantiated
----------------
0.000000
[-8073089, -8074096, -8076118, -8072759, -8078590, -8077501, -8079700, 8388607]
[96, -96, 4064]
----------------
----------------
1.000000
[-8073251, -8074257, -8076282, -8072926, -8078751, -8077665, -8079862, 8388607]
[0, 0, 0]
----------------
----------------
2.000000
[-8073652, -8074659, -8076683, -8073332, -8079159, -8078067, -8080270, 8388607]
[0, 0, 0]
----------------
----------------
3.000000
[-8073665, -8074671, -8076696, -8073342, -8079171, -8078082, -8080277, 8388607]
[0, 0, 0]
----------------
----------------
4.000000
[-8073419, -8074421, -8076446, -8073093, -8078921, -8077832, -8080024, 8388607]
[0, 0, 0]
----------------
----------------
5.000000
[-8073281, -8074292, -8076308, -8072957, -8078787, -8077699, -8079894, 8388607]
[0, 0, 0]
----------------
----------------
6.0000

----------------
120.000000
[-8080905, -8081914, -8083944, -8080581, -8086412, -8085322, -8087513, 8388607]
[0, 0, 0]
----------------
----------------
121.000000
[-8080592, -8081604, -8083633, -8080269, -8086094, -8085008, -8087199, 8388607]
[0, 0, 0]
----------------
----------------
122.000000
[-8080707, -8081718, -8083749, -8080387, -8086212, -8085117, -8087312, 8388607]
[0, 0, 0]
----------------
----------------
123.000000
[-8081188, -8082194, -8084227, -8080864, -8086692, -8085598, -8087797, 8388607]
[0, 0, 0]
----------------
----------------
124.000000
[-8081207, -8082212, -8084246, -8080879, -8086707, -8085616, -8087812, 8388607]
[0, 0, 0]
----------------
----------------
125.000000
[-8080894, -8081898, -8083927, -8080562, -8086387, -8085294, -8087492, 8388607]
[0, 0, 0]
----------------
----------------
126.000000
[-8080941, -8081950, -8083977, -8080610, -8086437, -8085344, -8087541, 8388607]
[144, -144, 8096]
----------------
----------------
127.000000
[-8081279, -8082283

----------------
240.000000
[-8088654, -8089666, -8091691, -8088326, -8094152, -8093064, -8095250, 8388607]
[0, 0, 0]
----------------
----------------
241.000000
[-8088686, -8089694, -8091719, -8088356, -8094182, -8093097, -8095279, 8388607]
[0, 0, 0]
----------------
----------------
242.000000
[-8088371, -8089377, -8091404, -8088042, -8093860, -8092779, -8094957, 8388607]
[0, 0, 0]
----------------
----------------
243.000000
[-8088485, -8089492, -8091521, -8088156, -8093978, -8092896, -8095073, 8388607]
[0, 0, 0]
----------------
----------------
244.000000
[-8088913, -8089923, -8091954, -8088585, -8094410, -8093328, -8095506, 8388607]
[80, -192, 8064]
----------------
----------------
245.000000
[-8088982, -8089986, -8092017, -8088651, -8094479, -8093394, -8095570, 8388607]
[0, 0, 0]
----------------
----------------
246.000000
[-8088727, -8089728, -8091762, -8088397, -8094222, -8093136, -8095315, 8388607]
[0, 0, 0]
----------------
----------------
247.000000
[-8088657, -8089659,

----------------
104.000000
[-8096012, -8097007, -8099047, -8095677, -8101489, -8100405, -8102576, 8388607]
[0, 0, 0]
----------------
----------------
105.000000
[-8096470, -8097467, -8099501, -8096136, -8101951, -8100866, -8103037, 8388607]
[0, 0, 0]
----------------
----------------
106.000000
[-8096467, -8097471, -8099501, -8096135, -8101953, -8100864, -8103041, 8388607]
[0, 0, 0]
----------------
----------------
107.000000
[-8096111, -8097114, -8099148, -8095777, -8101594, -8100503, -8102681, 8388607]
[128, -176, 8176]
----------------
----------------
108.000000
[-8096209, -8097211, -8099246, -8095872, -8101689, -8100599, -8102779, 8388607]
[0, 0, 0]
----------------
----------------
109.000000
[-8096591, -8097589, -8099630, -8096255, -8102075, -8100980, -8103165, 8388607]
[0, 0, 0]
----------------
----------------
110.000000
[-8096782, -8097781, -8099820, -8096447, -8102267, -8101175, -8103356, 8388607]
[0, 0, 0]
----------------
----------------
111.000000
[-8096488, -8097486

----------------
224.000000
[-8103432, -8104428, -8106472, -8103082, -8108902, -8107824, -8109978, 8388607]
[0, 0, 0]
----------------
----------------
225.000000
[-8103617, -8104615, -8106657, -8103274, -8109089, -8108011, -8110167, 8388607]
[0, 0, 0]
----------------
----------------
226.000000
[-8104202, -8105201, -8107246, -8103859, -8109677, -8108598, -8110756, 8388607]
[144, -192, 8128]
----------------
----------------
227.000000
[-8104292, -8105292, -8107334, -8103944, -8109771, -8108686, -8110846, 8388607]
[0, 0, 0]
----------------
----------------
228.000000
[-8103827, -8104827, -8106864, -8103479, -8109306, -8108221, -8110379, 8388607]
[0, 0, 0]
----------------
----------------
229.000000
[-8103894, -8104889, -8106927, -8103541, -8109367, -8108285, -8110439, 8388607]
[0, 0, 0]
----------------
----------------
230.000000
[-8104487, -8105482, -8107518, -8104136, -8109961, -8108879, -8111037, 8388607]
[0, 0, 0]
----------------
----------------
231.000000
[-8104650, -8105646

...

----------------
116.000000
[-8212107, -8213062, -8215173, -8211696, -8217457, -8216416, -8218422, 8388607]
[0, 0, 0]
----------------
----------------
117.000000
[-8211972, -8212931, -8215047, -8211564, -8217329, -8216283, -8218291, 8388607]
[0, 0, 0]
----------------
----------------
118.000000
[-8211485, -8212439, -8214556, -8211074, -8216836, -8215791, -8217796, 8388607]
[160, -176, 8176]
----------------
----------------
119.000000
[-8211759, -8212707, -8214827, -8211342, -8217106, -8216059, -8218071, 8388607]
[0, 0, 0]
----------------
----------------
120.000000
[-8212215, -8213167, -8215291, -8211802, -8217565, -8216519, -8218532, 8388607]
[0, 0, 0]
----------------
----------------
121.000000
[-8212214, -8213171, -8215294, -8211804, -8217567, -8216520, -8218533, 8388607]
[0, 0, 0]
----------------
----------------
122.000000
[-8211749, -8212704, -8214824, -8211336, -8217100, -8216052, -8218066, 8388607]
[0, 0, 0]
----------------
----------------
123.000000
[-8211845, -8212800

----------------
236.000000
[-8218421, -8219371, -8221494, -8218007, -8223766, -8222722, -8224722, 8388607]
[0, 0, 0]
----------------
----------------
237.000000
[-8218871, -8219819, -8221945, -8218456, -8224212, -8223176, -8225169, 8388607]
[128, -192, 8112]
----------------
----------------
238.000000
[-8218641, -8219591, -8221714, -8218221, -8223977, -8222941, -8224934, 8388607]
[0, 0, 0]
----------------
----------------
239.000000
[-8218089, -8219036, -8221156, -8217666, -8223421, -8222384, -8224377, 8388607]
[0, 0, 0]
----------------
----------------
240.000000
[-8218444, -8219388, -8221515, -8218023, -8223775, -8222741, -8224736, 8388607]
[0, 0, 0]
----------------
----------------
241.000000
[-8219011, -8219955, -8222085, -8218594, -8224344, -8223313, -8225304, 8388607]
[0, 0, 0]
----------------
----------------
242.000000
[-8218901, -8219845, -8221975, -8218487, -8224237, -8223202, -8225195, 8388607]
[0, 0, 0]
----------------
----------------
243.000000
[-8218415, -8219357

----------------
100.000000
[-8224727, -8225672, -8227805, -8224309, -8230063, -8229023, -8231011, 8388607]
[0, 0, 0]
----------------
----------------
101.000000
[-8225153, -8226099, -8228229, -8224737, -8230489, -8229451, -8231439, 8388607]
[0, 0, 0]
----------------
----------------
102.000000
[-8225706, -8226650, -8228780, -8225282, -8231034, -8229997, -8231991, 8388607]
[0, 0, 0]
----------------
----------------
103.000000
[-8225589, -8226533, -8228659, -8225165, -8230914, -8229881, -8231871, 8388607]
[0, 0, 0]
----------------
----------------
104.000000
[-8225095, -8226044, -8228170, -8224674, -8230425, -8229393, -8231378, 8388607]
[0, 0, 0]
----------------
----------------
105.000000
[-8225359, -8226306, -8228434, -8224939, -8230689, -8229653, -8231639, 8388607]
[0, 0, 0]
----------------
----------------
106.000000
[-8225870, -8226819, -8228947, -8225450, -8231201, -8230166, -8232152, 8388607]
[0, 0, 0]
----------------
----------------
107.000000
[-8225815, -8226769, -82288

----------------
221.000000
[-8231384, -8232324, -8234455, -8230951, -8236704, -8235659, -8237643, 8388607]
[0, 0, 0]
----------------
----------------
222.000000
[-8231786, -8232726, -8234857, -8231355, -8237107, -8236064, -8238049, 8388607]
[0, 0, 0]
----------------
----------------
223.000000
[-8232305, -8233249, -8235379, -8231879, -8237631, -8236591, -8238572, 8388607]
[0, 0, 0]
----------------
----------------
224.000000
[-8232245, -8233188, -8235319, -8231819, -8237565, -8236531, -8238509, 8388607]
[0, 0, 0]
----------------
----------------
225.000000
[-8231764, -8232710, -8234838, -8231337, -8237079, -8236049, -8238027, 8388607]
[0, 0, 0]
----------------
----------------
226.000000
[-8231997, -8232944, -8235076, -8231575, -8237311, -8236286, -8238267, 8388607]
[0, 0, 0]
----------------
----------------
227.000000
[-8232616, -8233556, -8235691, -8232184, -8237931, -8236902, -8238881, 8388607]
[0, 0, 0]
----------------
----------------
228.000000
[-8232528, -8233473, -82356

KeyboardInterrupt: 

#### Now we can add code to be able to plot the data

We will define a simple funtion live_plot to plot the raw data onto the notebook. We also need to define another fuction, let's call it "data_process", that takes in the raw data from the board, stores it in a global variable and allows us to dynamically change the plot as we get more signals and calls on the live_plot function. For this example let's plot only the first 5 channels.

In [26]:
def live_plot(sample, figsize=(17,16)):
    data = np.array(sample)
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    color = 'rgbycmkr'
    labels = ['Channel 1', 'Channel 2', 'Channel 3', 'Channel 4', 'Channel 5', 'Channel 6', 'Channel 7', 'Channel 8']
    
    try:
        for j in range(5):
            num = int('81'+str(j+1))
            plt.subplot(num)
            if j == 0:
                plt.title("EEG Data")
            plt.plot(data[-40:][:,j], label=labels[j], scaley=True, color=color[j])
            plt.autoscale(enable=True, axis='y', tight=None)
        
    except:
        for j in range(5):
            num = int('81'+str(j+1))
            plt.subplot(num)
            if j == 0:
                plt.title("EEG Data")
            plt.plot(data[-40:][:,j], label=labels[j], scaley=True, color=color[j])
            plt.autoscale(enable=True, axis='y', tight=None)
            
    plt.xlabel('time')
    plt.legend(loc='center left') # the plot evolves to the right
    plt.show();
    
    
bci_data = []

def data_process(sample):
    global bci_data
    Scale_Factor= (4.5/ 24) /(2^23 - 1) # specific for Cyton
    data = [i * Scale_Factor for i in sample.channel_data]
    bci_data.append(data)
    live_plot(bci_data)

Now let's run the code we just added.

In [31]:
if __name__ == '__main__':
    port = 'COM5'
    baud = 115200
    logging.basicConfig(filename="test.log", format='%(asctime)s -%(levelname)s : %(message)s', level=logging.DEBUG)
    logging.info('--------------LOG START ------------')
    board = bci.OpenBCICyton(port=port, scaled_output=False, log=True)
    print("Board Instantiated")
    board.ser.write(str.encode('v'))
    time.sleep(10)
    board.start_streaming(data_process)
    board.print_bytes_in()

KeyboardInterrupt: 

The graph is a bit slow, but this is a great way to start to understand how to use the OpenBCI data for your application.

#### Signal Processing:

The next steps on your OpenBCI Python journey would be to do some signal processing. [Here](https://github.com/J77M/openbciGui_filter_test/blob/master/gui_saved_data_filter.ipynb) is a good example of some of the filters you can use with your OpenBCI data.